In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from loan_analysis.loan import AmortizingLoan
from loan_analysis.interest import (
    IBORProcess,
    IBORGaussianProccess,
    get_loan_rates)
from loan_analysis.summary import Summary

from loan_analysis.utils import (
    plot_summary,
    plot_repayments,
    plot_mean_pm_std,
    plot_mean_and_95_percentile)
    
import ipywidgets as ipyw

Usage of the loan_analysis module comes in 3 steps:  

1. setting an `AmortizingLoan` with an initial principal and a termination time
2. instantiating an `IBORProcess` (such as `IBORGaussianProcess`) from which to sample Euribor traces, and fitting it to historical data  
3. running the simulation and using the `Summary` class to analyse the results

The following cell gives a demonstration of how to put this all together:

In [3]:
# simulations will have a signature something like this
def run_simulation_demo(
    loan: AmortizingLoan,
    ibor_process: IBORProcess,
    num_simulations: int=1000
    ) -> Summary:
    return Summary(
        values=np.random.randn(num_simulations),
        name="This Is Just A Demonstration")

# they can be wrapped in interactive tools with globals
# to minimize recomputation like this
loan = None
ibor_process = None
summary = None
def widget_function_demo(
    initial_principal,
    full_term,
    num_simulations,
    historical_ibor,
    plot_hist,
    plot_cdf,
    plot_kde,
    var_percent,
    ):
    global loan
    global ibor_process
    global summary
    recompute_summary = False
    if loan is None:
        print("initialising loan")
        loan = AmortizingLoan(
            initial_principal=initial_principal,
            full_term=full_term)
        recompute_summary = True
    elif (loan.initial_principal != initial_principal)\
        or (loan.full_term != full_term):
        print("re-initialising loan")
        loan = AmortizingLoan(
            initial_principal=initial_principal,
            full_term=full_term)
        recompute_summary = True
    if ibor_process is None:
        print("initialising ibor_process")
        ibor_process = IBORGaussianProccess(historical_ibor)
        recompute_summary = True
    elif len(ibor_process.historical_ibor) != len(historical_ibor):
        print("re-initialising ibor_process")
        ibor_process = IBORGaussianProccess(historical_ibor)
        recompute_summary = True
    if summary is None or recompute_summary:
        print("computing summary")
        summary = run_simulation_demo(loan, ibor_process, num_simulations)
    elif len(summary.values) != num_simulations:
        print("re-computing summary")
        summary = run_simulation_demo(loan, ibor_process, num_simulations)
    # print("DEMONSTRATION RESULT")
    fig = plt.figure(figsize=(12,5))
    plot_summary(
        summary,
        plot_cdf=plot_cdf,
        plot_kde=plot_kde,
        plot_hist=plot_hist,
        var_percent=var_percent,
        ax=plt.gca())
    plt.show()

widget = ipyw.interactive(
    widget_function_demo,
    initial_principal=ipyw.FloatText(value=1000),
    full_term=ipyw.IntText(value=400),
    num_simulations=ipyw.IntText(value=100),
    plot_hist=ipyw.Checkbox(value=True),
    plot_kde=ipyw.Checkbox(value=False),
    plot_cdf=ipyw.Checkbox(value=True),
    var_percent=ipyw.FloatSlider(value=5., min=0., max=100., step=0.5),
    historical_ibor=ipyw.fixed(np.cumsum(np.random.rand(100))))

commands = ipyw.HBox(
    children=widget.children[:-1],
    # layout={'flex-flow': 'flex-wrap'},
    layout=ipyw.Layout(flex_flow='row wrap')
    )
display(ipyw.VBox([commands, widget.children[-1]]))

# Euribor Curves

Okay, so now let's load our historical euribor and fit a `IBORGaussianProcess` model:

In [4]:
historical_euribor = pd.read_csv('euribor_1999_2016_12month.csv')
euribor_process = IBORGaussianProccess(
    historical_ibor=historical_euribor.rate.values/100,
    autocorrelation_maxlag=100,
    kernel_type="laplace")


def widget_function_euribor(
    kernel_type="laplace",
    maxlag=100,
    num_samples=2,
    min_rate=0.,
):
    global euribor_process
    if euribor_process.kernel_type != kernel_type \
    or euribor_process.autocorrelation_maxlag != maxlag:
        euribor_process = IBORGaussianProccess(
            historical_ibor=historical_euribor.rate.values/100,
            autocorrelation_maxlag=maxlag,
            kernel_type=kernel_type)
    samples = euribor_process.sample_ibor_curve(
        number_of_months=len(euribor_process.historical_ibor),
        num_samples=num_samples,
        initial_value=euribor_process.historical_ibor[0])
    samples[samples < min_rate] = min_rate
    fig = plt.figure(figsize=(12,4))
    plt.plot(
        euribor_process.historical_ibor,
        color='black', lw=4, label='historical data')
    for i in range(num_samples):
        plt.plot(
            samples[:,i],
            label='samples' if i == 0 else None)
    plt.legend(loc="upper left")
    plt.title("Sampled 12month Euribor Curves")
    plt.show()

widget = ipyw.interactive(
    widget_function_euribor,
    _interactive__options={'manual':True, 'manual_name':"Press to Run"},
    kernel_type=ipyw.Dropdown(
        options=['laplace', 'gaussian', 'mixture'],
        description="kernel type",
        layout=ipyw.Layout(width="200px")),
    num_samples=ipyw.IntText(
        value=2, min=0,
        description="no. samples",
        layout=ipyw.Layout(width="150px")),
    maxlag=ipyw.IntText(
        value=100, min=1,
        description="max lag",
        layout=ipyw.Layout(width="150px")),
    min_rate=ipyw.FloatText(
        value=0.,
        description="min rate",
        layout=ipyw.Layout(width="150px"))
        )
widget.manual_button.style.button_color = 'lightgreen'
widget.manual_button.style.font_weight = 'bold'

commands = ipyw.HBox(
    children=widget.children[:-2],
    layout=ipyw.Layout(flex_flow='row wrap'))
display(ipyw.VBox([widget.children[-2], commands, widget.children[-1]]))

What's going on here? It's a rather crude implementation of a Gaussian process. Instead of maximising the (log-)marginal likelihood (as explained [in this book](http://gaussianprocess.org/gpml/chapters/RW.pdf), for example), it fits a kernel to the autocorrelation of the data.  

The historical data (shown in black) has certain autocorrelations in it which we can call $\rho(t_1,t_2) = \rho(|t_2-t_1|)$, and a standard deviation $\sigma$.  

In `IBORGaussianProcess` the code fits either a Laplace kernel

$$\kappa^\text{laplace}_\tau(|t_2-t_1|)=\exp(-\tau\,|t_2-t_1|)$$

or a Gaussian kernel

$$\kappa^\text{gaussian}_\tau(|t_2-t_1|) = \exp(-\tau\,(t_2-t_1)^2)$$

or a $3$-parameter mixture

$$\kappa^\text{mixture}_{\tau_1,\tau_2,p}(|t_2-t_1|) =p\kappa^\text{gaussian}_{\tau_1}(|t_2-t_1|) + (1-p)\kappa^\text{laplace}_{\tau_2}(|t_2-t_1|)$$

to $\rho(t_1,t_2)$ by minimising the weighted mean squared error:  

$$
\tau^* = \argmin_{\tau} \sum_{\Delta t =0}^\text{max lag} (n-\Delta t)\left( \kappa_\tau(\Delta t) - \rho(\Delta t) \right)^2
$$

where $n$ is the number of observed historical Euribor rates. The weights are there to capture how the variance of the estimator or the true correlations shrinks in proportion to $1/(n-\Delta t)$ by the Law of Large Numbers.

These kernels can then be used to sample Gaussian processes, which are scaled by $\sigma$ and are then shifted to have a desired initial value, in this case the same as the historical data.

These samples are new estimates of Euribor curves with similar autocorrelations and standard deviation to those observed in the historical data.

# Repayment Schedules

For a given interest rate and principal amount, how much will be paid in interest?

First we look at the case of a fixed interest.

In [5]:
def widget_function_timecourse(
    initial_principal=231500.,
    full_term=37*12,
    fixed_interest_percent=3.286
    ):
    loan = AmortizingLoan(
        initial_principal=initial_principal,
        full_term=full_term)
    monthly_payment = loan.get_amortized_payment_amount(fixed_interest_percent/100)
    interest_rates_constant = np.full(loan.full_term, fixed_interest_percent/100)
    loan.pay_interest_rates(interest_rates_constant)
    fig = plt.figure(figsize=(12,4))
    plot_repayments(loan)
    plt.title(f"Calculated Monthly Cost: {monthly_payment:.2f} euros")
    plt.show()


widget = ipyw.interactive(
    widget_function_timecourse,
    initial_principal=ipyw.FloatSlider(
        min=0., max=400000., value=231500., step=1000.,
        description="amount borrowed"),
    full_term=ipyw.IntSlider(
        value=37*12, min=1, max=40*12, step=1,
        description="no. months"),
    fixed_interest_percent=ipyw.FloatSlider(
        value=3.286, min=0., max=20., step=0.01,
        description="interest %", readout_format='.3f')
    )
principal_text = ipyw.FloatText(
    value=231500.)
ipyw.link((principal_text, "value"), (widget.children[0], "value"))
full_term_text = ipyw.IntText(
    value=37*12)
ipyw.link((full_term_text, "value"), (widget.children[1], "value"))
interest_text = ipyw.FloatText(
    value=3.286)
ipyw.link((interest_text, "value"), (widget.children[2], "value"))
top_row_list = [principal_text, full_term_text, interest_text]
vboxes = [ipyw.VBox(list(xy)) for xy in zip(widget.children, top_row_list)]

top_row = ipyw.HBox(
    vboxes,
    layout=ipyw.Layout(flex_flow='row wrap'))
display(ipyw.VBox([
    top_row,
    widget.children[-1]]))

What happens if the interest rates are *flexible*, locked to the Euribor process from before?

**Note: TODO:** some weird error seems to arise when the full term is not a fixed number of years.

In [6]:
def widget_function_timecourse_stochastic(
    initial_principal=231500.,
    number_of_years=37,
    initial_interest_percentage=3.286,
    spread_percentage=1.,
    kernel_type="laplace",
    maxlag=100,
    num_samples=2,
    minimum_ibo_rate_percent=0.,
    individual_plots=True,
    fill_percent=95.
):
    global euribor_process
    if euribor_process is None:
        euribor_process = IBORGaussianProccess(
            historical_ibor=historical_euribor.rate.values/100,
            autocorrelation_maxlag=maxlag,
            kernel_type=kernel_type)
    elif euribor_process.kernel_type != kernel_type \
    or euribor_process.autocorrelation_maxlag != maxlag:
        euribor_process = IBORGaussianProccess(
            historical_ibor=historical_euribor.rate.values/100,
            autocorrelation_maxlag=maxlag,
            kernel_type=kernel_type)
    interest_rate_curves = euribor_process.sample_ibor_curve(
        number_of_months=number_of_years*12,
        num_samples=num_samples,
        initial_value=euribor_process.historical_ibor[0])
    interest_rate_curves += initial_interest_percentage/100 - interest_rate_curves[0,:]
    # rectify
    interest_rate_curves[interest_rate_curves < minimum_ibo_rate_percent/100] = minimum_ibo_rate_percent/100
    loan_rates = get_loan_rates(
        interest_rate_curves,
        spread=spread_percentage/100,
        renew_every=12
        )
    loan = AmortizingLoan(
        initial_principal=initial_principal,
        full_term=number_of_years*12)
    # monthly_payment = loan.get_amortized_payment_amount(fixed_interest_percent/100)
    # interest_rates_constant = np.full(loan.full_term, fixed_interest_percent/100)
    fig, axes = plt.subplots(2,1,figsize=(12,8))
    # ax = plt.gca()
    ax = axes[0]
    if individual_plots:
        for i in range(num_samples):
            loan.reset()
            loan.pay_interest_rates(loan_rates[:,i])
            ax = plot_repayments(
                loan,
                colour_interest=f"C{i}",
                colour_principal=f"C{i}",
                show_legend= i==0,
                ax=ax)
    else:
        interest_payments = np.zeros((number_of_years*12, num_samples))
        principal_payments = np.zeros((number_of_years*12, num_samples))
        for i in range(num_samples):
            loan.reset()
            loan.pay_interest_rates(loan_rates[:,i])
            interest_payments[:,i] = np.cumsum(loan.interests_paid)
            principal_payments[:,i] = np.cumsum(loan.principal_reductions)
        ax.axhline(initial_principal, ls=":", color='black')
        ax = plot_mean_and_95_percentile(
            principal_payments,
            colour="C1",
            label="cumulative principal paid",
            percentile=100.-fill_percent,
            ax=ax)
        ax = plot_mean_and_95_percentile(
            interest_payments,
            colour="C0",
            label="cumulative interest paid",
            percentile=100.-fill_percent,
            ax=ax)
        ax.set_xlabel("Month")
        ax.set_ylabel("Cumulative Payment (euros)")
        ax.grid(":")
        ax.legend(loc="upper left")
    if individual_plots:
        for i in range(num_samples):
            axes[1].plot(
                interest_rate_curves[:,i],
                label='ibor rate' if i == 0 else None,
                color=f"C{i}")
            axes[1].plot(
                loan_rates[:,i],
                label='loan rate' if i == 0 else None,
                ls="--",
                color=f"C{i}")
            axes[1].legend(loc="upper left")
    else:
        ax = axes[1]
        ax = plot_mean_and_95_percentile(
            interest_rate_curves,
            colour="C1",
            label="ibor rates",
            percentile=100.-fill_percent,
            ax=ax)
        ax = plot_mean_and_95_percentile(
            loan_rates,
            colour="C0",
            label="loan rates",
            percentile=100.-fill_percent,
            ax=ax)
        axes[1].legend(loc="upper left")
    plt.show()


# widget_function_timecourse_stochastic()
widget = ipyw.interactive(
    widget_function_timecourse_stochastic,
    _interactive__options={'manual':True, 'manual_name':"Press to Run"},
    initial_principal=ipyw.FloatSlider(
        min=0., max=400000., value=231500., step=1000.,
        description="amount borrowed"),
    number_of_years=ipyw.IntSlider(
        value=37, min=1, max=40, step=1,
        description="no. years"),
    initial_interest_percentage=ipyw.FloatSlider(
        value=0.992, min=0., max=20., step=0.01,
        description="initial %", readout_format='.3f'),
    spread_percentage=ipyw.FloatSlider(
        value=1.0, min=0., max=20., step=0.01,
        description="spread %", readout_format='.3f'),
    kernel_type=ipyw.Dropdown(
        options=['laplace', 'gaussian', 'mixture'],
        description="kernel type",
        layout=ipyw.Layout(width="200px")),
    num_samples=ipyw.IntText(
        value=2, min=0,
        description="no. samples",
        layout=ipyw.Layout(width="150px")),
    maxlag=ipyw.IntText(
        value=100, min=1,
        description="max lag",
        layout=ipyw.Layout(width="150px")),
    minimum_ibo_rate_percent=ipyw.FloatText(
        value=0.,
        description="min ibor %",
        layout=ipyw.Layout(width="150px")),
    individual_plots=ipyw.Checkbox(
        value=True, description="plot individual traces"),
    fill_percent=ipyw.FloatSlider(
        value=95., min=0., max=100., description="fill %")
    )
widget.manual_button.style.button_color = 'lightgreen'
widget.manual_button.style.font_weight = 'bold'
principal_text = ipyw.FloatText(
    value=231500.)
ipyw.link((principal_text, "value"), (widget.children[0], "value"))
years_text = ipyw.IntText(
    value=37)
ipyw.link((years_text, "value"), (widget.children[1], "value"))
interest_text = ipyw.FloatText(
    value=0.992)
ipyw.link((interest_text, "value"), (widget.children[2], "value"))
spread_text = ipyw.FloatText(
    value=1.)
ipyw.link((spread_text, "value"), (widget.children[3], "value"))
top_row_list = [principal_text, years_text, interest_text, spread_text]
vboxes = [ipyw.VBox(list(xy)) for xy in zip(widget.children, top_row_list)]

top_row = ipyw.HBox(
    vboxes,
    layout=ipyw.Layout(flex_flow='row wrap'))
ipyw.VBox([
    widget.manual_button,
    ipyw.HBox(
        [*widget.children[4:-2]],
        layout=ipyw.Layout(flex_flow='row wrap')),
    top_row,
    widget.children[-1]])

# Distributions

Next let's look at distributions of amounts paid and of maximum payments, or how often $p\%$ of payments are above an amount $x$.

Then we move onto more complex payment schedules, like starting with fixed payments, or paying above the asked amount.

In [7]:
# TODO: this cell can be neatened, and probably made interactive
from typing import Callable, Union, Optional

loan = None
def run_simulation(
    num_simulations: int=1000,
    maxlag: int=100,
    kernel_type: str='laplace',
    # number_of_years: int=37,
    initial_interest_percentage: float=0.992,
    minimum_ibo_rate_percent: float=-10.,
    spread_percentage: float=1.,
    # initial_principal: float=231500.,
    summary_functional: Callable[[AmortizingLoan],Union[np.ndarray,float]]=lambda loan: np.sum(loan.payments_made),
    summary_name: Optional[str]=None
    ) -> Summary:
    global euribor_process
    global loan
    if euribor_process is None:
        euribor_process = IBORGaussianProccess(
            historical_ibor=historical_euribor.rate.values/100,
            autocorrelation_maxlag=maxlag,
            kernel_type=kernel_type)
    elif euribor_process.kernel_type != kernel_type \
    or euribor_process.autocorrelation_maxlag != maxlag:
        euribor_process = IBORGaussianProccess(
            historical_ibor=historical_euribor.rate.values/100,
            autocorrelation_maxlag=maxlag,
            kernel_type=kernel_type)
    interest_rate_curves = euribor_process.sample_ibor_curve(
        number_of_months=loan.full_term,
        num_samples=num_simulations,
        initial_value=euribor_process.historical_ibor[0])
    interest_rate_curves += initial_interest_percentage/100 - interest_rate_curves[0,:]
    # rectify
    interest_rate_curves[interest_rate_curves < minimum_ibo_rate_percent/100] = minimum_ibo_rate_percent/100
    loan_rates = get_loan_rates(
        interest_rate_curves,
        spread=spread_percentage/100,
        renew_every=12
        )
    summary_values = []
    for i in range(num_simulations):
        loan.reset()
        loan.pay_interest_rates(loan_rates[:,i])
        summary_values.append(summary_functional(loan))
    return Summary(
        values=summary_values,
        name=summary_name)


summary = None
FUNCTIONALS = dict(
    total_spent=lambda loan: np.sum(loan.payments_made),
    highest_payment=lambda loan: np.max(loan.payments_made),
    percentile_95_payment=lambda loan: np.percentile(loan.payments_made, 95)
)
def widget_function_distribution(
    initial_principal=231500.,
    number_of_years=37,
    initial_interest_percentage=3.286,
    spread_percentage=1.,
    var_percent=5.,
    kernel_type="laplace",
    maxlag=100,
    num_samples=100,
    minimum_ibo_rate_percent=0.,
    summary_variable='total_spent',
    plot_cdf=True,
    plot_kde=False,
    plot_hist=True,
    # individual_plots=True,
    # fill_percent=95.
):

    global euribor_process
    global summary
    global loan
    # recompute_summary = False
    recompute_summary = True  # always recompute
    if euribor_process is None:
        euribor_process = IBORGaussianProccess(
            historical_ibor=historical_euribor.rate.values/100,
            autocorrelation_maxlag=maxlag,
            kernel_type=kernel_type)
        recompute_summary = True
    elif euribor_process.kernel_type != kernel_type \
    or euribor_process.autocorrelation_maxlag != maxlag:
        euribor_process = IBORGaussianProccess(
            historical_ibor=historical_euribor.rate.values/100,
            autocorrelation_maxlag=maxlag,
            kernel_type=kernel_type)
        recompute_summary = True
    if loan is None:
        loan = AmortizingLoan(
            initial_principal=initial_principal,
            full_term=number_of_years*12)
        recompute_summary = True
    elif loan.initial_principal != initial_principal \
    or loan.full_term != number_of_years*12:
        loan = AmortizingLoan(
            initial_principal=initial_principal,
            full_term=number_of_years*12)
        recompute_summary = True
    
    summary_functional = FUNCTIONALS.get(summary_variable)
    # if summary is None:
    #     recompute_summary = True
    # elif len(summary.values) != num_samples:
    #     recompute_summary = True
    # elif summary.name != summary_variable:
    #     recompute_summary = True
    if recompute_summary:
        summary = run_simulation(
            num_simulations=num_samples,
            maxlag=maxlag,
            kernel_type=kernel_type,
            # number_of_years=number_of_years,
            initial_interest_percentage=initial_interest_percentage,
            minimum_ibo_rate_percent=minimum_ibo_rate_percent,
            spread_percentage=spread_percentage,
            # initial_principal=initial_principal,
            summary_functional=summary_functional,
            summary_name=summary_variable)

    fig = plt.figure(figsize=(12,5))
    plot_summary(
        summary,
        plot_cdf=plot_cdf,
        plot_kde=plot_kde,
        plot_hist=plot_hist,
        var_percent=var_percent,
        ax=plt.gca())
    plt.grid(ls=":")
    plt.show()

widget = ipyw.interactive(
    widget_function_distribution,
    _interactive__options={'manual':True, 'manual_name':"Press to Run"},
    initial_principal=ipyw.FloatSlider(
        min=0., max=400000., value=231500., step=1000.,
        description="amount borrowed"),
    number_of_years=ipyw.IntSlider(
        value=37, min=1, max=40, step=1,
        description="no. years"),
    initial_interest_percentage=ipyw.FloatSlider(
        value=0.992, min=0., max=20., step=0.01,
        description="initial %", readout_format='.3f'),
    spread_percentage=ipyw.FloatSlider(
        value=1.0, min=0., max=20., step=0.01,
        description="spread %", readout_format='.3f'),
    var_percent=ipyw.FloatSlider(value=5., min=0., max=100., step=0.5),
    kernel_type=ipyw.Dropdown(
        options=['laplace', 'gaussian', 'mixture'],
        description="kernel type",
        layout=ipyw.Layout(width="200px")),
    num_samples=ipyw.IntText(
        value=100, min=0,
        description="no. samples",
        layout=ipyw.Layout(width="150px")),
    maxlag=ipyw.IntText(
        value=100, min=1,
        description="max lag",
        layout=ipyw.Layout(width="150px")),
    minimum_ibo_rate_percent=ipyw.FloatText(
        value=0.,
        description="min ibo %",
        layout=ipyw.Layout(width="150px")),
    summary_variable=ipyw.Dropdown(options=FUNCTIONALS.keys()),
    plot_hist=ipyw.Checkbox(value=True),
    plot_kde=ipyw.Checkbox(value=False),
    plot_cdf=ipyw.Checkbox(value=True),
    )
widget.manual_button.style.button_color = 'lightgreen'
widget.manual_button.style.font_weight = 'bold'
principal_text = ipyw.FloatText(
    value=231500.)
ipyw.link((principal_text, "value"), (widget.children[0], "value"))
years_text = ipyw.IntText(
    value=37)
ipyw.link((years_text, "value"), (widget.children[1], "value"))
interest_text = ipyw.FloatText(
    value=0.992)
ipyw.link((interest_text, "value"), (widget.children[2], "value"))
spread_text = ipyw.FloatText(
    value=1.)
ipyw.link((spread_text, "value"), (widget.children[3], "value"))
var_text = ipyw.FloatText(
    value=5.)
ipyw.link((var_text, "value"), (widget.children[4], "value"))
top_row_list = [principal_text, years_text, interest_text, spread_text, var_text]
vboxes = [ipyw.VBox(list(xy)) for xy in zip(widget.children, top_row_list)]

top_row = ipyw.HBox(
    vboxes,
    layout=ipyw.Layout(flex_flow='row wrap'))
ipyw.VBox([
    widget.manual_button,
    ipyw.HBox(
        [*widget.children[len(top_row_list):-2]],
        # [*widget.children[4:-1]],
        layout=ipyw.Layout(flex_flow='row wrap')),
    top_row,
    widget.children[-1]])

# Other Stuff..

* use bqplot instead of matplotlib, or even bokeh, for more interactive plots
* use scikit-learn gaussian process regressor, or fit with maximum likelihood ([see this book](http://gaussianprocess.org/gpml/chapters/RW.pdf))  
  * learn more about Gaussian processes for machine learning
* create SDE models of interest rate curves